In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Echarts Jupyter lab support
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [ ]:
# Echarts imports
from pyecharts import options as opts
from pyecharts.globals import  ThemeType
from pyecharts.charts import Line

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt

In [ ]:
from common.log_helper import LogHelper
from backtest.scale_backtest_args import ScaleBacktestArgGen
from backtest.scale_backtest import ScaleBacktest
from modelservice.alpha_consts import SignalDirection
from notebooks.echart_bt_plots import *
from notebooks.echart_df_plot import *
import logging
import sys
import pandas as pd

In [ ]:
#LogHelper.configure_logging(verbose=False)
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
def colorize(val):
    if val == 'B':
        color = 'green'
    elif val == 'S':
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color

In [ ]:
BtArgs = {
    'common': {
        'start_time': '09:15:01',
        'end_time' : '15:24:30',
        # 'start_time': '13:20:01',
        # 'end_time' : '13:30:01',
    },
    'alpha': {
        'sigmult' : 0.5,
        'smoother' : {
            'TS' : {'enabled': False, 'type' : 'STABLEWIN' , 'timeperiod' : 3 * 60 },
            'UL' : {'enabled': True, 'type' : 'EMA' , 'timeperiod' : 30 },
            'CP' : {'enabled': True, 'type' : 'SMA' , 'timeperiod' : 30 },            
            'E3' : {'enabled': True, 'timeperiod' : 13 * 60 },                        
            'MACD' : {'enabled': True, 'dfast' : 12*3*60, 'dslow' : 26*3*60, 'dsig' : 9*3*60 },                        
        }
    },
    'execution': {
        'exec_instr_type' : 'NK_F',        
        'price_type' : 'CROSS', 
        'opt_trd_offset' : 2,
        'quote_lots' : 1,
        'stop_loss': 0,
        'max_loss' : 0,
        'profit_target' : 0
    }
}

In [ ]:
dump_base_dir = "/home/qoptrprod/data/dumps/"
trading_date = "20240419"
underlying = "NIFTY_I"

In [ ]:
arg_obj = ScaleBacktestArgGen.resolve_and_gen_args(
    dump_base_dir=dump_base_dir,
    underlying=underlying,
    trading_date=trading_date,
    bt_args=BtArgs
)
bt = ScaleBacktest(arg_obj)
result = bt.run_bt()
trade_df = pd.DataFrame(result['trademarkers'])

In [ ]:
#print(result)

In [ ]:
def buy_or_sell(v):
    if v == 'BUY':
        return 'color : lightgreen'
    elif v == 'SELL':
        return 'color : red'
    elif v == 'OPEN':        
        return 'background-color : darkorange'   
    elif v == None:
        return ''
    return ''

In [ ]:
#trade_df.style.map(lambda x: buy_or_sell(x)).format({col: "{:.1f}" for col in ['price','profit','cost','to','sigval', 'cpx','ppx','ulpx', 'atm']})

In [ ]:
sum_df = pd.DataFrame([result['summary']])
sum_df

### Signal Run To For Detailed Plots

In [ ]:
sig_args_obj = arg_obj
bt = ScaleBacktest(sig_args_obj)
result = bt.run_signal()

In [ ]:
signal_df = pd.DataFrame(result)

In [ ]:
signal_df.head(2)

In [ ]:
signal_mark_points = []
color_green = 'rgba(50,250,50,0.7)'
color_red = 'rgba(250,50,50,0.7)'
for _, row in signal_df.iterrows():
    color = color_green
    if row.dir == SignalDirection.BUY:
        color = color_green
    elif row.dir == SignalDirection.SELL:
        color = color_red
    else:
        continue # skip dir = None
    signal_mark_points.append(opts.MarkPointItem(coord=[row.ulexts, row.ulpx], 
            #value=row.val,
            symbol='circle',               
            symbol_size=[8, 8],
            itemstyle_opts=opts.ItemStyleOpts(color=color),
            ))

In [ ]:
macd_mark_points = []
color_green = 'rgba(70,250,70,0.8)'
color_red = 'rgba(250,70,70,0.8)'
last_dir = SignalDirection.NONE
for _, row in signal_df.iterrows():
    color = color_green
    if last_dir == row.macd_dir:
        continue
    if row.macd_dir == SignalDirection.BUY:
        color = color_green
    elif row.macd_dir == SignalDirection.SELL:
        color = color_red
    else:
        continue # skip dir = None
    last_dir = row.macd_dir    
    macd_mark_points.append(opts.MarkPointItem(coord=[row.ulexts, row.ulpx + 10], 
            #value=row.val,
            symbol='square',               
            symbol_size=[5, 5],
            itemstyle_opts=opts.ItemStyleOpts(color=color,),
                                                       
            ))

In [ ]:
trade_plot = plot_trade_signals(signal_df, 
                                trade_df, 
                                sum_df, 
                                show_pnl_markers=False,
                                show_legends=True, 
                                signal_mark_points=None,
                                mark_offset=2)

In [ ]:
trade_plot.load_javascript()

In [ ]:
trade_plot.render_notebook()

### Signal Plot

In [ ]:
signal_plot = plot_df_simple(
    signal_df, 
    x_col = 'ulexts', 
    y_cols = ['ulpx' , 'smooth_ulpx', 'atm'], 
    mark_points = {
        'ulpx' :  signal_mark_points + macd_mark_points,
    },
    title= f"Signal Analysis - {underlying} {trading_date}",
    zoom=True
)
cp_plot = plot_df_simple(
    signal_df, 
    x_col = 'ulexts', 
    y_cols = ['cpx' , 'ppx'], 
    title= f"Call Put - {underlying} {trading_date}"
)
macd_plot = plot_df_simple(
    signal_df, 
    x_col = 'ulexts', 
    y_cols = ['ulpx'], 
    mark_points = {'ulpx' :  macd_mark_points},
    title= f"MACD - {underlying} {trading_date}",
    zoom=True
)

In [ ]:
signal_plot.load_javascript()
cp_plot.load_javascript()
macd_plot.load_javascript()

In [ ]:
signal_plot.render_notebook()

In [ ]:
cp_plot.render_notebook()

In [ ]:
macd_plot.render_notebook()